In [2]:
using XLSX
using DataFrames, TimeSeries
using StatsPlots
using Plots
using LinearAlgebra
using CSV
using StatsBase
using GLM
include("utils.jl")
include("hmm_est.jl")

regime_summary (generic function with 1 method)

In [205]:
#columns = [:Date, :USA_Net, :World_ex_USA, :EM_Net, :ACWI_Net]
columns = [:Date, :USA_Net, :World_ex_USA, :EM_Net, :ACWI_Net]
assets = [:USA_Net, :World_ex_USA, :EM_Net]
data_df = DataFrame(XLSX.readtable("data/assets_data_03.24.xlsx", "Indexes"; infer_eltypes=true))[2:end, columns]
dropmissing!(data_df, disallowmissing=true)
returns_usd = data_df |> 
            x -> TimeArray(x; timestamp = :Date) |> 
            x -> percentchange(x, :log)

435×4 TimeArray{Float64, 2, Date, Matrix{Float64}} 1988-01-31 to 2024-03-31
┌────────────┬────────────┬──────────────┬─────────────┬─────────────┐
│            │ USA_Net    │ World_ex_USA │ EM_Net      │ ACWI_Net    │
├────────────┼────────────┼──────────────┼─────────────┼─────────────┤
│ 1988-01-31 │  0.0410122 │    0.0153553 │   0.0935834 │   0.0246253 │
│ 1988-02-29 │  0.0401109 │    0.0640862 │   0.0031132 │   0.0560778 │
│ 1988-03-31 │ -0.0349774 │    0.0592025 │     0.10175 │   0.0302388 │
│ 1988-04-30 │ 0.00818061 │    0.0140195 │   0.0517417 │   0.0125024 │
│ 1988-05-31 │ 0.00760205 │   -0.0326806 │   0.0292525 │  -0.0201209 │
│ 1988-06-30 │  0.0445733 │   -0.0229028 │   0.0331365 │ -0.00119203 │
│ 1988-07-31 │ -0.0040323 │    0.0288082 │  -0.0260317 │     0.01818 │
│ 1988-08-31 │  -0.035096 │   -0.0667177 │  -0.0274362 │  -0.0566908 │
│     ⋮      │     ⋮      │      ⋮       │      ⋮      │      ⋮      │
│ 2023-09-30 │ -0.0483353 │   -0.0342956 │  -0.0264995 │   -0.042229 │
│

In [206]:
period_returns = returns_usd[assets] |> x -> from(x, Date(1987, 12,1 )) |> x -> to(x, Date(2022,09,30))

417×3 TimeArray{Float64, 2, Date, Matrix{Float64}} 1988-01-31 to 2022-09-30
┌────────────┬─────────────┬──────────────┬─────────────┐
│            │ USA_Net     │ World_ex_USA │ EM_Net      │
├────────────┼─────────────┼──────────────┼─────────────┤
│ 1988-01-31 │   0.0410122 │    0.0153553 │   0.0935834 │
│ 1988-02-29 │   0.0401109 │    0.0640862 │   0.0031132 │
│ 1988-03-31 │  -0.0349774 │    0.0592025 │     0.10175 │
│ 1988-04-30 │  0.00818061 │    0.0140195 │   0.0517417 │
│ 1988-05-31 │  0.00760205 │   -0.0326806 │   0.0292525 │
│ 1988-06-30 │   0.0445733 │   -0.0229028 │   0.0331365 │
│ 1988-07-31 │  -0.0040323 │    0.0288082 │  -0.0260317 │
│ 1988-08-31 │   -0.035096 │   -0.0667177 │  -0.0274362 │
│     ⋮      │      ⋮      │      ⋮       │      ⋮      │
│ 2022-03-31 │     0.03417 │    0.0115373 │  -0.0228443 │
│ 2022-04-30 │  -0.0952756 │   -0.0679247 │  -0.0572341 │
│ 2022-05-31 │ -0.00270982 │    0.0082682 │   0.0043937 │
│ 2022-06-30 │  -0.0868359 │   -0.0988188 │  -0.068761

In [216]:
[assets; :ACWI_Net]

4-element Vector{Symbol}:
 :USA_Net
 :World_ex_USA
 :EM_Net
 :ACWI_Net

In [217]:
period_returns = returns_usd[[assets; :ACWI_Net]] |> x -> from(x, Date(1987, 12,1 )) |> x -> to(x, Date(2022,09,30))
returns_summarystats(period_returns, 12)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
USA_Net,0.1489,1.4788,0.0928,0.4192,-2.257,-0.718,-0.1985,0.0114,1.4606,0.6231,0.1528
World_ex_USA,0.1684,1.7129,0.0461,0.4125,-2.7986,-0.6393,-0.2748,0.0626,1.6545,0.2735,0.0946
EM_Net,0.2253,2.0825,0.0854,0.5562,-4.1,-0.9239,-0.3119,0.1546,3.1476,0.379,0.1224
ACWI_Net,0.1538,1.3949,0.0672,0.3834,-2.65,-0.81,-0.2172,0.0528,2.0316,0.4371,0.1424


In [208]:
hmm = hmm_est(period_returns, 3)
regime_summary(hmm[1], assets, 36)

Means
Standard deviations
Regimes transition matrix


Regime,USA_Net,World_ex_USA,EM_Net
1,0.381,0.289,0.317
2,0.090,-0.191,0.019
3,0.319,0.328,0.597


Regime,USA_Net,World_ex_USA,EM_Net
1,0.201,0.227,0.299
2,0.355,0.376,0.550
3,0.152,0.269,0.168


"Correlations in regime 1"

,USA_Net,World_ex_USA,EM_Net
USA_Net,1.000,0.837,0.680
World_ex_USA,0.837,1.000,0.837
EM_Net,0.680,0.837,1.000


"Correlations in regime 2"

,USA_Net,World_ex_USA,EM_Net
USA_Net,1.000,0.834,0.680
World_ex_USA,0.834,1.000,0.784
EM_Net,0.680,0.784,1.000


"Correlations in regime 3"

,USA_Net,World_ex_USA,EM_Net
USA_Net,1.000,0.040,0.551
World_ex_USA,0.040,1.000,-0.322
EM_Net,0.551,-0.322,1.000


Regime,1,2,3
1,0.975,0.025,0.000
2,0.043,0.768,0.189
3,0.000,0.476,0.524


In [220]:
n_assets = length(assets)
scenarios = simulate_hmm(hmm[1], n_assets, 180, 100_000)

for a in 1:n_assets
    print_percentiles(scenarios[a,:,:], [0.02, 0.1, 0.3, 0.5, 0.7, 0.9, 0.98], 12, string(assets[a]))
end 

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,-0.3103,-0.2794,-0.2674,-0.2582,-0.2568,-0.2539,-0.247,-0.2483,-0.2485,-0.2496,-0.248,-0.2487,-0.2488,-0.2486,-0.2455
0.1,-0.163,-0.1342,-0.119,-0.1107,-0.106,-0.1046,-0.103,-0.1016,-0.1023,-0.1024,-0.1013,-0.0998,-0.1019,-0.1028,-0.1018
0.3,-0.026,-0.0004,0.011,0.016,0.0191,0.0213,0.0214,0.021,0.0214,0.0207,0.0218,0.0223,0.0215,0.0223,0.0225
0.5,0.0655,0.0838,0.0916,0.0949,0.0975,0.099,0.0987,0.0991,0.0984,0.0984,0.0989,0.0995,0.0992,0.0994,0.0996
0.7,0.1539,0.1648,0.1681,0.1704,0.1723,0.1726,0.1727,0.1732,0.172,0.1719,0.1722,0.1742,0.1731,0.1724,0.1734
0.9,0.2826,0.2807,0.2793,0.2782,0.2783,0.2793,0.2787,0.2782,0.2782,0.2779,0.2768,0.2791,0.2785,0.2765,0.2783
0.98,0.4171,0.4021,0.398,0.3937,0.3928,0.3916,0.3908,0.3895,0.391,0.3901,0.3912,0.3892,0.3905,0.3893,0.3901


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,-0.4214,-0.385,-0.3735,-0.3609,-0.3563,-0.3547,-0.3519,-0.3473,-0.3502,-0.353,-0.3532,-0.3497,-0.3494,-0.3498,-0.3489
0.1,-0.2582,-0.2192,-0.2019,-0.1912,-0.1863,-0.1819,-0.1813,-0.1782,-0.1811,-0.1817,-0.1794,-0.1805,-0.1795,-0.1812,-0.1814
0.3,-0.1016,-0.0648,-0.0494,-0.0433,-0.037,-0.035,-0.0347,-0.0349,-0.0335,-0.0352,-0.034,-0.0336,-0.0352,-0.034,-0.0344
0.5,0.0036,0.0325,0.0453,0.0491,0.0533,0.0551,0.0553,0.0556,0.0552,0.0555,0.0546,0.056,0.0552,0.0543,0.0559
0.7,0.1042,0.1258,0.1329,0.1374,0.1388,0.14,0.1394,0.1405,0.1405,0.1399,0.1388,0.1401,0.1402,0.1388,0.141
0.9,0.2486,0.2566,0.2573,0.2578,0.258,0.2589,0.2587,0.2579,0.2566,0.2583,0.2571,0.2587,0.2593,0.2575,0.2578
0.98,0.395,0.3893,0.3851,0.3858,0.3808,0.3833,0.3818,0.3806,0.3807,0.3835,0.3784,0.3813,0.3836,0.3794,0.38


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,-0.5058,-0.4553,-0.4407,-0.429,-0.4222,-0.4187,-0.4129,-0.4114,-0.4144,-0.4125,-0.4124,-0.4084,-0.4097,-0.4091,-0.411
0.1,-0.2767,-0.2339,-0.2163,-0.2068,-0.2022,-0.2014,-0.1977,-0.1972,-0.1984,-0.2003,-0.1981,-0.1974,-0.1993,-0.1975,-0.1999
0.3,-0.0721,-0.0404,-0.0315,-0.027,-0.0238,-0.022,-0.0223,-0.0211,-0.0214,-0.0233,-0.0224,-0.0202,-0.0226,-0.0221,-0.0221
0.5,0.0677,0.082,0.086,0.0877,0.0891,0.0912,0.0902,0.0919,0.0901,0.0902,0.09,0.0922,0.0901,0.0895,0.0909
0.7,0.2031,0.2029,0.2006,0.1999,0.2001,0.2001,0.2008,0.2003,0.2002,0.1999,0.2007,0.2012,0.2009,0.1988,0.1997
0.9,0.3984,0.3794,0.3701,0.3682,0.3648,0.3634,0.3638,0.3635,0.363,0.3621,0.3625,0.3629,0.3628,0.3611,0.3626
0.98,0.6058,0.5759,0.5584,0.5488,0.5518,0.5489,0.5466,0.5449,0.5455,0.5491,0.5444,0.5476,0.543,0.5418,0.5465


In [218]:
weights = [0.321, 0.67, 0.009]
#weights = [.7, .3]
acwi_cum_ret = cumprod(scenarios .+ 1, dims=3)

acwi = zeros(100_000, 180)
for t in 1:180
    acwi[:,t] = acwi_cum_ret[:,:,t]' * weights * 100
end

acwi = acwi[:, 2:end] ./ acwi[:,1:end-1] .-1

print_percentiles(expm1.(acwi),[0.05, 0.1, 0.3, 0.5, 0.7, 0.9, 0.95], 12)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0.05,-0.256,-0.2287,-0.2121,-0.2033,-0.1964,-0.1941,-0.191,-0.188,-0.1862,-0.1849,-0.1854,-0.1849,-0.1845,-0.1824
0.1,-0.1878,-0.1599,-0.1429,-0.1352,-0.1296,-0.1261,-0.1242,-0.1216,-0.1201,-0.1187,-0.1202,-0.1187,-0.1189,-0.1177
0.3,-0.0495,-0.0231,-0.0109,-0.0038,-0.0016,0.0022,0.0036,0.0037,0.0051,0.006,0.0055,0.0071,0.0076,0.0074
0.5,0.0435,0.0643,0.0722,0.0786,0.0797,0.0826,0.0838,0.0836,0.0846,0.0856,0.086,0.0864,0.0867,0.0861
0.7,0.1326,0.1478,0.1524,0.1565,0.1574,0.1598,0.1604,0.1592,0.1615,0.1605,0.1623,0.1628,0.1628,0.1618
0.9,0.2643,0.2668,0.2666,0.2688,0.2679,0.2695,0.27,0.2691,0.2705,0.2689,0.2705,0.2718,0.2725,0.2708
0.95,0.3271,0.3255,0.3229,0.3237,0.3213,0.3238,0.3227,0.3233,0.3246,0.3212,0.3238,0.3244,0.3249,0.3243


In [6]:
M = [1 2 1; 3 4 1; 5 6 1]
cov(M')

3×3 Matrix{Float64}:
 0.333333  0.666667  1.0
 0.666667  2.33333   4.0
 1.0       4.0       7.0

In [8]:
M2 = [1 2 1; 3 4 1; 5 6 missing]
cov(M2')

MethodError: MethodError: Cannot `convert` an object of type Missing to an object of type Float64

Closest candidates are:
  convert(::Type{Float64}, !Matched::Measures.AbsoluteLength)
   @ Measures C:\Users\matsz\.julia\packages\Measures\PKOxJ\src\length.jl:12
  convert(::Type{T}, !Matched::DualNumbers.Dual) where T<:Union{Real, Complex}
   @ DualNumbers C:\Users\matsz\.julia\packages\DualNumbers\5knFX\src\dual.jl:24
  convert(::Type{T}, !Matched::Ratios.SimpleRatio{S}) where {T<:AbstractFloat, S}
   @ Ratios C:\Users\matsz\.julia\packages\Ratios\FsiCW\src\Ratios.jl:51
  ...
